In [4]:
# encoding: utf-8
import os
import numpy as np
import pandas as pd
import random
import math
from numpy import array
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend as K
# config = tf.ConfigProto()
# config.gpu_options.allow_growth=True
# sess = tf.Session(config=config)
# K.set_session(sess)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True

sess = tf.Session(config=config)

from tensorflow.nn.rnn_cell import LSTMStateTuple

from scipy.io import loadmat, savemat

In [5]:
base = ['vnf label', 'vnf min cpu', 'vnf min mem', 'vnf min sto', 'vnf vm cpu',
       'vnf vm mem', 'vnf vm sto', 'vnf pm id', 'vnf cpu usage',
       'vnf mem usage', 'vnf sto usage', 'vnf before', 'vnf after',
       'vnf label.1', 'vnf min cpu.1', 'vnf min mem.1', 'vnf min sto.1',
       'vnf vm cpu.1', 'vnf vm mem.1', 'vnf vm sto.1', 'vnf pm id.1',
       'vnf cpu usage.1', 'vnf mem usage.1', 'vnf sto usage.1', 'vnf before.1',
       'vnf after.1', 'sla', 'fg id', 'flow traffic', 'flow latency',
       'flow bnd usage', 'flow packet loss', 'min cpu affinity',
       'min mem affinity', 'min sto affinity', 'conflicts affinity',
       'real_affinity', 'static_affinity', 'predicted_affinity',
       'prediction_time', 'TF', 'MTT', 'MTT_upper', 'MTT_lower', 'MTTF_R',
       'MTTF_EM', 'MTTF_C', 'MTTF_TDDB', 'MTTF_SM', 'MTTFF_TC', 'A', 'AEM',
       'AC', 'ATDDB', 'ASM', 'ATC', 'TAA', 'QRED', 'QR', 'PUE', 'DCie', 'cost',
       'TIMESTAMP', 'EXTERNAL_TEMP', 'ROOM_TEMP', 'MTTF_IC', 'A_TC', 'Q_DIT',
       'TPF', 'AIRFLOW', 'TAAF', 'DeltaT_de', 'QD']
       
       
#Select your global variable among one of above, and place it after base below


base = 'vnf cpu usage'


In [6]:
fgs = pd.read_csv('../fgs.csv')

In [7]:
fgs.head()

,,,,4963594775,1,2
proxy.9.5698,traffic counter.7.5664,0.971442,82.243099,1.794354,9.341767,100.0
5402488769,5,1.000000,NaN,NaN,NaN,NaN
traffic counter.7.7413,proxy.9.11244,4.060322,119.086081,16.548350,8.474562,150.0
proxy.9.11244,proxy.9.2362,0.268318,23.865256,11.573841,9.248256,150.0
proxy.9.2362,traffic counter.7.9524,2.961259,80.083566,18.280812,4.783467,150.0


In [8]:
vns = pd.read_csv('../vnfs.csv', header=None)

In [9]:
vnf_id = np.array(vns[vns.columns[0]])
vnf_type = np.array(vns[vns.columns[1]])
vnf_scheduling = np.array(vns[vns.columns[2]])
vnf_pm = np.array(vns[vns.columns[3]])
vnf_fg = np.array(vns[vns.columns[4]])
flavor_data = np.array(vns[vns.columns[5:8]])
vm_data = np.array(vns[vns.columns[8:11]])
usage_data = np.array(vns[vns.columns[11:15]])

In [10]:
vm_data.shape

(12322, 3)

In [11]:
output = pd.read_csv('../new_results_out.csv', sep =';')#header=None)
output.drop('Unnamed: 0',axis = 1, inplace=True)

In [12]:
output.head()

,vnf label,vnf min cpu,vnf min mem,vnf min sto,vnf vm cpu,vnf vm mem,vnf vm sto,vnf pm id,vnf cpu usage,vnf mem usage,...,EXTERNAL_TEMP,ROOM_TEMP,MTTF_IC,A_TC,Q_DIT,TPF,AIRFLOW,TAAF,DeltaT_de,QD
0,dpi.2.136144,0.035914,0.226393,0.001757,0.06250,0.12720,0.001930,257345416,71.88000,1.75780,...,15,23,2435.249666,0.996726,1.839678,14.999377,0.131858,0.872117,6.132259,6.132259
1,traffic counter.7.40368,0.063818,0.017674,0.000223,0.06189,0.03821,0.000386,3488796635,3.51600,6.34800,...,15,24,49.077683,0.859840,1.801941,14.999402,0.142946,0.768427,6.006469,6.006469
2,proxy.9.13456,0.018672,0.020912,0.000006,0.01250,0.01590,0.000404,4820294844,0.18044,0.13352,...,55,25,82.372536,0.911478,1.800100,54.999420,0.152569,0.757640,6.000332,6.000332
3,firewall.3.36933,0.140469,0.021706,0.000068,0.09375,0.03198,0.000070,3550430599,26.46000,6.44600,...,40,23,0.044220,0.005497,1.814606,39.999481,0.132889,0.807089,6.048686,6.048686
4,dpi.2.92594,0.015708,0.050654,0.000810,0.06250,0.08569,0.000926,4820072706,41.74000,14.42800,...,21,29,0.011200,0.001398,1.823040,20.999427,0.189031,0.790021,6.076802,6.076802


In [13]:
output.columns

Index(['vnf label', 'vnf min cpu', 'vnf min mem', 'vnf min sto', 'vnf vm cpu',
       'vnf vm mem', 'vnf vm sto', 'vnf pm id', 'vnf cpu usage',
       'vnf mem usage', 'vnf sto usage', 'vnf before', 'vnf after',
       'vnf label.1', 'vnf min cpu.1', 'vnf min mem.1', 'vnf min sto.1',
       'vnf vm cpu.1', 'vnf vm mem.1', 'vnf vm sto.1', 'vnf pm id.1',
       'vnf cpu usage.1', 'vnf mem usage.1', 'vnf sto usage.1', 'vnf before.1',
       'vnf after.1', 'sla', 'fg id', 'flow traffic', 'flow latency',
       'flow bnd usage', 'flow packet loss', 'min cpu affinity',
       'min mem affinity', 'min sto affinity', 'conflicts affinity',
       'real_affinity', 'static_affinity', 'predicted_affinity',
       'prediction_time', 'TF', 'MTT', 'MTT_upper', 'MTT_lower', 'MTTF_R',
       'MTTF_EM', 'MTTF_C', 'MTTF_TDDB', 'MTTF_SM', 'MTTFF_TC', 'A', 'AEM',
       'AC', 'ATDDB', 'ASM', 'ATC', 'TAA', 'QRED', 'QR', 'PUE', 'DCie', 'cost',
       'TIMESTAMP', 'EXTERNAL_TEMP', 'ROOM_TEMP', 'MTTF_IC', 'A_T

In [14]:
output[base].shape

(195379,)

In [15]:
output = output[:12322]
output = output[base]

In [16]:
thelist = [vnf_id, vnf_type, vnf_scheduling, vnf_pm, vnf_fg, flavor_data, vm_data, usage_data]
for i in range(len(thelist)):
    print(thelist[i].shape)
    if len(thelist[i].shape) == 1:
        thelist[i] = np.reshape(thelist[i], (-1, 1))


(12322,)
(12322,)
(12322,)
(12322,)
(12322,)
(12322, 3)
(12322, 3)
(12322, 4)


In [17]:
vnf_id = np.reshape(vnf_id, (-1, 1))
vnf_type = np.reshape(vnf_type, (-1, 1))
vnf_scheduling = np.reshape(vnf_scheduling, (-1, 1))
vnf_pm = np.reshape(vnf_pm, (-1, 1))
vnf_fg = np.reshape(vnf_fg, (-1, 1))

In [18]:
vnf_id.shape

(12322, 1)

In [19]:
inputt = np.concatenate((vnf_id, vnf_type, vnf_scheduling, vnf_pm, vnf_fg, flavor_data, vm_data, usage_data), axis=1)

In [20]:
inputt.shape

(12322, 15)

In [21]:
trainx, testx, trainy, testy = train_test_split(inputt, output, test_size=0.18)

In [22]:
print("Trainx shape", trainx.shape)
print("Trainy shape", trainy.shape)
print("Testx shape", testx.shape)
print("Testy shape", testy.shape)

Trainx shape (10104, 15)
Trainy shape (10104,)
Testx shape (2218, 15)
Testy shape (2218,)


In [23]:
np.gcd(10104, 2218)

2

In [24]:
trainx = np.array(trainx)
trainy = np.array(trainy)
testx = np.array(testx)
testy = np.array(testy)

In [25]:
#Reshaping data for LSTM

trainx = trainx.reshape((-1, 2, 15))
trainy = trainy.reshape((-1, 2))
testx = testx.reshape((-1, 2, 15))
testy = testy.reshape((-1, 2))

## Starting Construction of model

In [26]:
tf.reset_default_graph()
no_units = 50


#### Making Placeholders

In [27]:
train = tf.placeholder(dtype=tf.float32, shape=[None, 2, 15])
target = tf.placeholder(dtype = tf.float32, shape = [None, 2])
batch_size = 50

In [28]:
dataset = tf.data.Dataset.from_tensor_slices((train, target)).batch(batch_size).repeat()
test_dataset = tf.data.Dataset.from_tensors((train, target)).repeat()
iterr = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
inputt, outputt = iterr.get_next()

#Creating Initialization operations
train_init_op = iterr.make_initializer(dataset)
test_init_op = iterr.make_initializer(test_dataset)

W0726 11:18:28.102379 140527127572608 deprecation.py:323] From <ipython-input-28-805daaeb55b3>:3: DatasetV1.output_types (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
W0726 11:18:28.103054 140527127572608 deprecation.py:323] From <ipython-input-28-805daaeb55b3>:3: DatasetV1.output_shapes (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
W0726 11:18:28.108556 140527127572608 deprecation.py:323] From /usr/local/lib/python3.5/site-packages/tensorflow/python/data/ops/iterator_ops.py:348: Iterator.output_types (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
W0726 11:18:28.109143 140527127572608 depreca

In [29]:
n_batches = math.ceil(trainx.shape[0]/batch_size)

### Making Forward and Backward Structure

In [30]:
lstm_fw = [tf.nn.rnn_cell.LSTMCell(num_units = no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]
lstm_bw = [tf.nn.rnn_cell.LSTMCell(num_units = no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]

for i in range(2):
    lstm_fw.append(tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]))
    lstm_bw.append(tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]))
    

W0726 11:18:29.621497 140527127572608 deprecation.py:506] From /usr/local/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0726 11:18:29.624227 140527127572608 deprecation.py:323] From <ipython-input-30-9000bcacb521>:1: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0726 11:18:29.625959 140527127572608 deprecation.py:323] From <ipython-input-30-9000bcacb521>:5: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is e

In [31]:
encoder_outputs, encoder_fw_state, encoder_bw_state = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                                                    cells_fw=lstm_fw,
                                                    cells_bw = lstm_bw,
                                                    inputs = inputt,
                                                    dtype = tf.float32)


W0726 11:18:31.516993 140527127572608 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0726 11:18:31.520122 140527127572608 deprecation.py:323] From /usr/local/lib/python3.5/site-packages/tensorflow/contrib/rnn/python/ops/rnn.py:239: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
W0726 11:18:31.521269 140527127572608 deprecation.py:323] From /usr/local/lib/python3.5/site-packages/tensorflow/python/ops/rnn.py:464: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecat

In [32]:
encoder_state_c = tf.concat((encoder_fw_state[-1][0].c, encoder_bw_state[-1][0].c), 1)

encoder_state_h = tf.concat((encoder_fw_state[-1][0].h, encoder_bw_state[-1][0].h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_state_c,
    h=encoder_state_h
)

In [33]:
output = tf.layers.Dense(2)(encoder_state_h)

In [34]:
loss = tf.losses.mean_squared_error(outputt, output)
train_op = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

W0726 11:18:32.969033 140527127572608 deprecation.py:323] From /usr/local/lib/python3.5/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [35]:
if not os.path.exists('tmp'):
    os.makedirs('tmp')

In [36]:
epochs = 10

In [37]:
losx = []
saver = tf.train.Saver()

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    if(tf.train.checkpoint_exists('tmp/model_tensorflow.ckpt')):
        saver.restore(sess, 'tmp/model_tensorflow.ckpt')
        print("Model Restored")
#     sess.run(iter.initializer, feed_dict={train: x, target: y})
    sess.run(train_init_op, feed_dict={train: trainx, target: trainy})
    for i in range(epochs):
        for _ in range(n_batches):
                lossx, idk = sess.run([loss, train_op])
                losx.append([loss])                   
        print("Current epoch going on is...",i,"and current loss is...", lossx)
    save_path = saver.save(sess, 'tmp/model_tensorflow.ckpt')
    print("Model saved in path: %s"%save_path)
    
    print("Now commencing, testing")
    sess.run(test_init_op, feed_dict = {train:testx, target:testy})
    print("Test loss: {:4f}".format(sess.run(loss)))

W0726 11:18:36.671230 140527127572608 deprecation.py:323] From <ipython-input-37-b69629eb4301>:6: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

2 root error(s) found.
  (0) Not found: Key dense/bias/Adam not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-37-b69629eb4301>:2) ]]
  (1) Not found: Key dense/bias/Adam not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-37-b69629eb4301>:2) ]]
	 [[save/RestoreV2/_7]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'save/RestoreV2':
  File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/l

In [ ]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    if(tf.train.checkpoint_exists('tmp/model_tensorflow.ckpt')):
        saver.restore(sess, 'tmp/model_tensorflow.ckpt')
        print("Model Restored")
#     sess.run(iter.initializer, feed_dict={train: x, target: y})
    sess.run(test_init_op, feed_dict = {train:testx, target:testy})
    print("Test loss: {:4f}".format(sess.run(loss)))
    prediction = sess.run(output)

In [ ]:
prediction.shape